<h1>Regressie (jaar)</h1>
begin met basics van maand regressie

In [ ]:
import pandas as pd
import pyodbc
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Maak verbinding met de Microsoft Access-database
conn_str_sales = (
    r"Driver={Microsoft Access Driver (*.mdb, *.accdb)};"
    r"DBQ=C:\\Users\\Burto\\Downloads\\GO-databases DEDS-week 4 & 5\\HC code\Week 10\\GO-databases schoon\\go_sales_schoon.accdb;"
    r"ExtendedAnsiSQL=1;"
)
conn_SALES = pyodbc.connect(conn_str_sales)

# Haal gegevens op uit de database
df_returnedItems = pd.read_sql_query("SELECT * FROM returned_item", conn_SALES)
df_details = pd.read_sql_query("SELECT * FROM order_details", conn_SALES)
df_header = pd.read_sql_query('SELECT * FROM order_header', conn_SALES)

#JOIN tabel
df = pd.merge(df_returnedItems, df_details, on='ORDER_DETAIL_CODE')
df = pd.merge(df,df_header, on='ORDER_NUMBER')

#Selecteer alleen de relevante informatie
df = df.dropna()
df = df[['ORDER_DATE','RETURN_QUANTITY']]

<h2>Voeg maanden toe</h2>

In [ ]:
# Zet ORDER_DATE om naar maanden
df['MONTH'] = pd.to_datetime(df['ORDER_DATE']).dt.month

# Groepeer gegevens per maand
df_grouped = df.groupby(['MONTH']).sum().reset_index()

<h2>Regressiemodel</h2>
<p>We kunnen beginnen met het toevoegen van een nieuwe kolom met het jaartal van de bestelling en het verwijderen van de ORDER_DATE-kolom</p>

In [ ]:
# Gebruik LINEAR REGRESSION om de relatie tussen maand en geretourneerde hoeveelheid te bepalen
X = df_grouped['MONTH'].values.reshape(-1, 1)
y = df_grouped['RETURN_QUANTITY'].values.reshape(-1, 1)
model = LinearRegression()
model.fit(X, y)

# Voorspel het aantal geretourneerde producten voor de maand januari
predicted_returns = model.predict([[1]])
print(predicted_returns)

<h3>Bereken R2 score</h3>
<p>Een hogere score is beter.<br/>Laten we de code testen door 2024 in te vullen, kijken wat er uit komt.</p>

In [ ]:
# Bereken de R2-score
y_pred = model.predict(X)
r2 = r2_score(y, y_pred)

# Print de de R2-score
print(r2)

<h2>Polynomial poging</h2>

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X)

model_poly = LinearRegression()
model_poly.fit(X_poly, y)
